In [1]:
%pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [65]:
import simpy
import random
import statistics
from collections import deque

In [123]:
# Params and constants. In minutes

#Global Simpy env variables
SEED = 1234
SIM_TIME = 2 * 60 # 8 hours of operation

#Variables for project analysis
#PAX_ARRIVALS = 1/5 # The expo rate passengers arrive at the monorail station.
HEADWAY = 5 # Cyclical minutes between each train's arrival. Constant set by monorail's operators
DWELL_TIME = 3 # Minutes that a train remains in the station to allow for passenger loading. Constant set by monorail's operators
CAR_CAPACITY = 10 # Number of passengers that the park's model of trains car can hold.
CARS_PER_TRAIN = 3 # Number of train cars coupled together

# Schedule to define peak times throughout park open hours, in 30 min intervals
# Assume empiral data collection determined a peak in the morning at opening time, then again around lunch when people leave and come back. It tapers out like a tail to end the day
arrival_schedule = {1: 15, 2: 12, 3: 10, 4: 5, # First two hours, 8AM to 10AM
                    5: 3, 6: 4, 7: 3, 8: 6, # Next two hours, 10AM to 12PM
                    9: 25, 10: 22, 11: 14, 12: 8, # Next two hours, 12PM to 2PM
                    13: 6, 14: 4, 15: 3 , 16: 1# Final two hours, 2PM to4PM
                    }


In [ ]:
platform_queue = []

In [80]:
def passenger_arrivals(env):
    """ Simpy function to generate passenger arrivals at the station platform and add to the FIFO queue"""
    pax_counter = 0

    
    while True:
        #Determine which interval the clock is in, 30 min intervals
        interval = (env.now // 30) + 1
        pax_interarrival = random.expovariate(arrival_schedule[interval])
        pax_counter += 1

        arrival_data = (pax_counter, env.now)
        pax_queue.append(arrival_data)
        
        yield env.timeout(pax_interarrival)
        mins, seconds = divmod(env.now * 60, 60) #this time format only for printing
        print(f"Passenger {pax_counter} arrived at t={int(mins)}:{int(seconds):02d} mins")

        

In [132]:
def pax_loading(env, headway=HEADWAY, capacity=CAR_CAPACITY, n_cars=CARS_PER_TRAIN):
    """ Will remove people from the queue and load the train at a stochastic rate.
    Will also have logic for handling max capacity/max loading time
    """

    print(f"train loading starts at {env.now:.2f}")
    for i in range(5):
        yield env.timeout(i/10)
        print(f"loaded {i+1}  @{env.now:.2f}")
    print(f"train loading ends at {env.now:.2f}")

In [115]:
def train_arrival(env, headway=HEADWAY, capacity=CAR_CAPACITY, n_cars=CARS_PER_TRAIN):
    """ Simpy function to create the train arrivals at determined cadence"""
    total_capacity = capacity * n_cars
    train_counter = 0
    while True:
        
        yield env.timeout(HEADWAY)
        train_counter += 1
        
        env.process(pax_loading(env, total_capacity))
        
        mins, seconds = divmod(env.now * 60, 60)  #this time format only for printing
        print(f"{n_cars} Car train with capacity {total_capacity} pax arrived at t={int(mins)}:{int(seconds):02d}")

In [133]:

#Stat collection data structures
#   more research to be done. just setting up a framework

pax_queue = deque() # FIFO queue to manage people arriving at and leaving in order. Stores pax id and arrival timestamp. Eg (id, time) or (4, 1.32)
pax_on_trains = [] # Store 



random.seed(SEED+1)
env = simpy.Environment()


#Processes
env.process(passenger_arrivals(env))
env.process(train_arrival(env))


env.run(until=SIM_TIME)

#Stat collection

Passenger 1 arrived at t=0:09 mins
Passenger 2 arrived at t=0:11 mins
Passenger 3 arrived at t=0:19 mins
Passenger 4 arrived at t=0:21 mins
Passenger 5 arrived at t=0:22 mins
Passenger 6 arrived at t=0:27 mins
Passenger 7 arrived at t=0:32 mins
Passenger 8 arrived at t=0:38 mins
Passenger 9 arrived at t=0:39 mins
Passenger 10 arrived at t=0:39 mins
Passenger 11 arrived at t=0:46 mins
Passenger 12 arrived at t=0:49 mins
Passenger 13 arrived at t=0:53 mins
Passenger 14 arrived at t=0:57 mins
Passenger 15 arrived at t=1:03 mins
Passenger 16 arrived at t=1:10 mins
Passenger 17 arrived at t=1:17 mins
Passenger 18 arrived at t=1:22 mins
Passenger 19 arrived at t=1:26 mins
Passenger 20 arrived at t=1:31 mins
Passenger 21 arrived at t=1:40 mins
Passenger 22 arrived at t=1:40 mins
Passenger 23 arrived at t=1:43 mins
Passenger 24 arrived at t=1:57 mins
Passenger 25 arrived at t=2:00 mins
Passenger 26 arrived at t=2:00 mins
Passenger 27 arrived at t=2:10 mins
Passenger 28 arrived at t=2:11 mins
P